# Seisberry configuration notes
Revision 2020-04-16

# Introduction
The goal is to build a 3 component seismograph (seismometer) based on a Raspberry Pi, an Analog to Digital Hat and 3 coil geophones.

# Requirments
The projects had to comply with the following technical requirments:
- 24 bit sampling for high dynamic and so gain setting is not critical in most applications.  
- capable of a sampling rate of 1,000 Hz or faster (1ms sample Interval). 
- Jitter at 1,000Hz inferior to 5%.
- 4 channels to operate with 3 geophones + 1 hydrophone (for a possible Ocean Bottom Node like use case, for very shallow water use).
- Output to Segy and Miniseed format for respectivly industrial and research purpose.
- price tag under $100 for a single component mode and under $150 for a 3 component land node, to be relevant to the education community and to be accessible in developping countries.
- All software opensource and 100% available to the community (Github).
- Possibility to operate the node in the field as a mobile unit, with the clock synchronized to GPS time.
- Possibility to operate the node as a permanent station, fully autonomous, and automatically uploading its data to a DMC remote server (Iris).
- One click access to daily recorded data, when the node is connected to a network, with results displayed on a dynamic web page hosted on the node.  

The first 4 requirements insure the node performs as well (or better) than most system available in the industry in 2020.

## Parts needed
- Raspberry Pi 3 B+: USD 35 (Raspberry Pi 4 has a higher power consumption and possible compatibility issue with the ADDA board): 
- Onboard ADS1256, 8 channels 24 bit high-precision ADC (4ch differential input), 30ksps sampling rate: USD 29
- 3 geophones (1 vertical, 2 horizontal), with their shunt resistance. I use: RTC-10Hz 395 Ohms with 1000 Ohms shunt: USD 30x3.
- wires and a box.


## Raspberry Pi OS install:
It is strongly suggested to use a Raspian OS as many useful libraries for Raspberry PI come pre-installed.

Download Raspian Buster (2020) image, check SHA 256 hash.
With Balena Etcher, flash the SD card with the Raspian Buster image.
Connect a screen, mouse and keyboard to the Pi.
Boot the Raspberry with the image out of Etcher.
Configure password (use a strong password as we will be opening SSH, VSP etc..) then the hostname to "seisberry".
Update the distribution (just follow the wizard that should pop up automatically after the first install).
Configure wifi, keyboard, date.
Allow VSP and SSH connect in Preference > Raspberry Pi Configuration.
Do ifconfig in a terminal and write down the node's IP on the local network. Example IPv4 is: 192.168.1.118
Power down.


## Board Setting:
Connect the High-Precision AD-DA Board to the Raspberry Pi, via the GPIO header.

Jumper settings:
Set the Power Supply to 5V: connect the pin 5V and VCC.
Set the Reference Input Voltage to 5V: connect the pin 5V and VREF.

For testing the board only:
Set the Potentiometer output as an Analog Input: connect the pin ADJ and AD0. Make sure the left side Sensor Interface AD0 is disconnected.
Set the LDR (Light Detection Resistor) output as an Analog Input: connect the pin LDR and AD1. Make sure the left side Sensor Interface AD1 is disconnected.
Last connect AINCOM to AGND, to set the board to single handed (as opposed to differential). 

Note: later, when using AD for differential measurements (which is what we will do for the geophone application), INCOM and AGND need to be disconnected, ie the jumper removed. Of couse LDR and Potentiometer will also be disconnected.


## VNC connect:
Power up the Raspberry Pi with the HAT now connected. Both the HAT and the Pi should have their power led on.
VNC to seisberry (I use VNC viewer, free). If your local DNS is setup right, "seisberry" is the address, alternativly use the IP previously noted. 
If you do not want to VNC, connect a screen, mouse and keyboard to the Pi.

The doc on sharewave (https://www.waveshare.com/wiki/High-Precision_AD/DA_Board) is out of date, applies for older versions for which the files are unavailable. Do not follow it, follow this tutorial instead: 


## GPIO libraries:

A powerful feature of the Raspberry Pi is the row of GPIO (general-purpose input/output) pins along the top edge of the board. A 40-pin GPIO header is found on all current Raspberry Pi boards. Any of the GPIO pins can be designated (in software) as an input or output pin and used for a wide range of purposes. Your High-Precision AD-DA Board is connected to the Pi's GPIO bus.


### RPi.GPIO and WiringPI:
WiringPi is a C library used by C programs for GPIO connections. 
RPi.GPIO is a Python library used by Python programs for GPIO connections.

Both libraries are PRE-INSTALLED with standard Raspbian systems! You do not need to install anything. 
Disregard the High-Precision AD-DA Board manual.

Optional step: you can check your GPIO libraries versions:
        

In [ ]:
gpio -v

and in Python:

In [ ]:
import RPi.GPIO as GPIO
GPIO.VERSION

### Bcm2835 libray:

This is a C GPIO library and this step could be skipped if you intent to use the board with Python only. 
However the sampling speed needed for the seismograph project will require C. 
Note I would have prefered to use the standard C GPIO library (WiringPi), installed on Raspian by default. However the existing C program we will reuse here is built on another GPIO library: BCM2835. So that's what we will use and install now. 
If somebody finds the time to port to WiringPi instead, please let me know.

Visit:  http://www.airspayce.com/mikem/bcm2835/ to get the latest version, or get a waveshare tested version from: 
https://www.waveshare.com/wiki/File:Bcm2835-1.45.tar.gz

Install bcm2835:

In [ ]:
tar zxvf bcm2835-1.xx.tar.gz
cd bcm2835xx

in the bcm2835xxx directory (Note: it seems this needs to be done as root to avoid the /dev/mem access problem further down the line):

In [ ]:
./configure
make
sudo make check
sudo make install

## Config I2C interface:

As well as simple input and output devices, the GPIO pins can be used with a variety of alternative functions, some are available on all pins, others on specific pins.
- PWM (pulse-width modulation)
- SPI (Serial Peripheral Interface)
- I2C (Inter-Integrated Circuit)
- Serial

The bcm2835 C library uses I2C to read digital inputs and setting digital outputs. 
For Python the I2C part could be skipped, as we are using spidev with SPI, Spidev is installed by default on Raspian. 
Note: Wiring Pi would also need I2C (or SPI).

Here is how we configure I2C for our board:


In [ ]:
sudo raspi-config

Interface Options -> I2C -> yes

In [ ]:
sudo nano /etc/modules

Add the following two lines to the configuration file:

In [ ]:
i2c-bcm2708
i2c-dev

Press the keys Ctrl+X to exit, and input Y to save the settings. 

Reboot the Raspberry PI
Select Interface Options -> I2C -> yes, to start up I2C core driver.

Check with:

In [ ]:
ls /dev/*i2c*

/dev/i2c-1

## Config SPI interface:

For both Python (as spidev uses SPI) and some C libraries:

In [ ]:
sudo raspi-config

Select Interface Options -> SPI -> yes, to start up SPI core driver.

Check with:

In [ ]:
ls /dev/*spi*

You should get:
/dev/spidev0.0  /dev/spidev0.1
These represent SPI devices on chip enable pins 0 and 1, respectively. These pins are hardwired within the Pi.


## Python:

Let's do some testing with Python first, to make sure everything is running smoothly.

The only dependencies are:
spidev, RPi.GPIO and some standard stuff. All are present by default on the Raspian, so you should no have any dependency problem. Spidev needs the SPI driver up, as done in the previous step.

Run main.py to test the board:

With the python script up and running:  
Block the LDR (Light Detection Resistor) from the light and then the voltage of channel AD1 will be changed.
Turn the potentiometer and the voltage of channel AD0 will be changed.
In the end, press Ctrl+C to suspend the current process.

Congratulation, you now have a functional Rapsberry Pi with a working AD Board, interfacing with Python.


## Setting the Jumpers to differential
Set the board to differential mode by physicaly removing the AINCOM to AGND jumper
At this point we are still in test mode measuring the LDR & potentiometer.
Why differential mode? If some eletromagnetic noise is present in your environment, it will be the same on both your inputs. By taking the difference (differential mode) the noise is removed. For details see:
https://en.wikipedia.org/wiki/Common-mode_rejection_ratio

You can rerun the Python test if you will.

## C:
Dependencies:
Standard libraries and bcm2835.h Needs I2C up and running.

The original C source comes from:
https://github.com/will127534/RaspberryPi-seismograph
And the original author discuss the project here: 
https://will-123456.blogspot.com/2019/04/diy-seismograph.html

I forked this project on Github to add a few features I needed, like the ability to take a parameter file as input, so the AD gain, processing gain, sample rate, and output path can be passed at execution in a parameter text file, and changing those does not necessitate to recompile the program. If no parameter file is passed as argument, the program defaults to the exact same behaviour as the original program.
My fork is available here, and it is what we'll be using from here: 
https://github.com/erellaz/RaspberryPi-seismograph

Here is an example of a parameter file:
- AmpGain : 64
- NumGain : 1.0
- PSP     : 750
- Path    : /media/pi/92ED-675B/


Multithreading:
If the same thread writes a chunck of data and then switches to the next chunck, the time spent spent in between causes some data to be lost. So the program has 2 threads, one reads from the card, then pass back to the main thread to write. Files are switched through the pipe.

Install:
Go to the obj directory of the diy-seismograph project and build the executable for your system and libs:

In [ ]:
cd RaspberryPi-seismograph-master/Software/RaspberryCode
make

Execute the compiled program:

In [ ]:
./ads1256_test

If you get the /dev/mem segmentation fault, it is indicative of an install problem with Bcm2835. A quick and dirty fix is to run the sampling progam setuid root:

In [ ]:
sudo chown root.root ads1256_test
sudo chmod +s ads1256_test

You should get a file like:
20200415_063041.txt
year month day _ hour minute second, one file every 15mn

In [ ]:
more 20200415_063041.txt
1586950241.002875810,0.078125000,0.078125000,0.078125000,001147643
1586950241.004261419,0.078125000,0.078125000,0.078125000,001353578
1586950241.005647236,0.078125000,0.078125000,0.078125000,001363005

With the following format:

In [ ]:
Epoch Time in sec   ,CH0	,CH1	    ,CH2	,SI in sec
1586950241.007033157,0.078125000,0.078125000,0.078125000,001364411
1586950241.008420277,0.078125000,0.078125000,0.078125000,001366234

Congratulation, your board is now running in differential mode with both C and Python.


## Geophones:

Measure resistance of your geophones (open circuit): I get 395 Ohms for my RTC-10Hz 395
Solder a 1,000 Ohm resistance in shunt + 2 wires. 
It is best practice to twist your pairs to reduce crosstalk and electromangnetic interference. 
https://en.wikipedia.org/wiki/Twisted_pair

Measure the new resistance of 283.15 Ohm (1/283.15)=(1/1000)+(1/395)  
On the board, remove the 2 jumpers connecting AD1 to LDR and ADO to ADJ: we are done with test mode, so we can disconnect the Photoresistor and Potentiometer.

Wifi is so high frequency it should not interfere with the relativly low frequencies we are trying to measure, however... at this point I recommend to power off the Wifi adapter of the Raspberry Pi board to reduce electromagnetic interference:

sudo iwconfig wlan0 txpower off

Use: 

sudo iwconfig wlan0 txpower on 

to enable it again if needed.

Connect the 3 geophones to the 6 AD pins in differential mode: Pins(0-1, 1-2, 3-4)

Start recording.

In [ ]:
./ads1256_test

Perform a geophone tap test to check response and polarity: 
the SEG standard for causal seismic data specifies that the onset of a compression from an explosive source is represented by a negative number, that is, by a downward deflection when displayed graphically.

You can do a top to check performance. The process relativly CPU intensive.

Congratulation, you have made a functional seismograph!



## Install Obspy
See here for details:
https://github.com/obspy/obspy/wiki/Installation-on-Linux-via-Apt-Repository

In [ ]:
sudo chmod 777 /etc/apt/sources.list

Add the following line at the end of the update file, assuming you use Raspian Buster:

In [ ]:
deb http://deb.obspy.org buster main

Get the public key for package integrity:

In [ ]:
wget --quiet -O - https://raw.githubusercontent.com/obspy/obspy/master/misc/debian/public.key | sudo apt-key add -

Install with dependencies:

In [ ]:
sudo apt-get update
sudo apt-get install python-obspy python3-obspy

then you can use plotgen.py to look at the data.


## Transfer data to and from seisberry:
Make sure ssh is enable in Preference> Raspberry Pi Config
then from a terminal (do command in a windows machine, or open a term in Linux or Mac)

In [ ]:
sftp pi@seisberry

then 
cd ... 
mget * or mget -r *


## Installing GPS for time source

Optional, only usefull if you have an USB GPS and intent to use the seisberry in the field, deconnected from the internet, and accurate timing is important for your application.
https://photobyte.org/raspberry-pi-stretch-gps-dongle-as-a-time-source-with-chrony-timedatectl/

# Processing the seismic data
Use the daily processing Python script. Latest versiosn available here:
https://github.com/erellaz/seisberry

There are 2 versions:
- one version optimized for hardware light systems, like the seisberry, it is prefixed Pi_. Rapsberries are 32 bit systems, limiting the max size of a numpy array to 2G. So the Rapsberry version of the script processes only one channels at a time and implements various strategies to be memory light. It runs without problem for 24 hours of production. It needs to be run for every channel (for 3 channels the script runs 3 times).
- one version optimized for 64 bit desktops. This version processes all channels at the same time. This is the one to use when you are processing the data on a desktop.

The scripts works in UTM, which is the standard in seismology.
The start of the script has some user updatable variables fully documented. Modify that, and only that, according to your location and goals.

The script runs in 4 stages:
-  connect to the USGS server to download the official list of earthquakes in the last 7 days. Filter those earthquakes by location and magnitude to retain only the one you have a change to see at your location, store them in a catalog, for labelling purposes.
- go though the production (1 file every 15mn), implement a robust reading strategy (some files may be missformed), and store the everything in a numpy array. If the numpy array already exists (rerun of the script), reads that instead, which is much faster.
- compute various statistics and diagnostics
- convert the data to numpy format and plot a "day plot", label with the catalog of official events.

# Making the seisberry a web server showing the day plots
First install Apache web server on the seisberry:

In [ ]:
sudo apt install apache2 -y

Then from any computer on your local network, go to: 
    http://seisberry
You should see the Debian Apache default Web page saying "It worked"

The default webpage is here: /var/www/html/index.html

Now we give ourself right to the directory:

In [ ]:
sudo usermod -a -G www-data pi
sudo chown -R -f www-data:www-data /var/www/html

Now we can make a gallery from our day plots:
Download the Pi_Make_gallery.py script from Github:
https://github.com/erellaz/seisberry

This is a very simple script making a web page showcasing the different plots generated by the previous script. It is quite simple and minimalist, but works well.

# Cleaning old data automatically
At 1ms of sample rate, and stored in text files, raw data piles up very quickly. I use 3 mini sd cards in my seisberry: one for the Raspian OS, one for the raw files, and one for the miniseed. Then a 3rd Python scripts cleans the older data automatically, after a few days, giving the user ample time to download the most interesting parts of the collected data.
Pi_daily_clean.py can be daonloaded at: https://github.com/erellaz/seisberry

# Upload your data to Iris
Register your station at Iris and obtain a host and port name to upload from for your station.

Download miniseed2dmc:
https://github.com/iris-edu/miniseed2dmc

Install it:

In [ ]:
make

Create a bash miniseed2dmc upload command.

In [ ]:
miniseed2dmc host:port data

# Automate all your tasks with crontab
Now we will use crontab to schedule every day the script generating the plot, then the script generating the webpage, then the script cleaning the sd card with the data. 

In [ ]:
crontab -e

Here is what my crontab looks like, (result of crontab -l)

In [ ]:
# m h  dom mon dow   command
0 05 * * * python3 /home/pi/Desktop/Daily/Daily_processing.py 1
0 06 * * * python3 /home/pi/Desktop/Daily/Daily_processing.py 2
0 07 * * * python3 /home/pi/Desktop/Daily/Daily_processing.py 3
0 08 * * * python3 /home/pi/Desktop/Daily/Make_gallery.py
0 09 * * * python3 /home/pi/Desktop/Daily/Daily_clean.py /media/pi/MINISEED_DATA mseed
4 09 * * * python3 /home/pi/Desktop/Daily/Daily_clean.py /var/www/html png
9 09 * * * python3 /home/pi/Desktop/Daily/Daily_clean.py /media/pi/RAW_DATA done


Rebbot the Raspberry and you are all set.